<a href="https://colab.research.google.com/github/jiminAn/2023_SUMMER_SAMSUNG_DL/blob/main/%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8C/Day4/Day_4_3_RNN_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 4.3 RNN Basic
### 실습 목표
- RNN 모델의 입력 데이터 형태를 살펴보고 모델을 정의해보자

### Contents
1. RNN 모델의 입력 데이터 형태
2. RNN 모델 정의
  1. vanila RNN
  2. LSTM


## 0. Setting
- DL 관련 library 호출
- GPU 설정

In [1]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


## 1. RNN 모델의 입력 데이터 형태

### 예시: "hello"를 RNN 모델의 입력 데이터로 넣어보자.
![img.png](https://3863425935-files.gitbook.io/~/files/v0/b/gitbook-legacy-files/o/assets%2F-LIA3amopGH9NC6Rf0mA%2F-M4bJ-IWAKzglR0XHFwU%2F-M4bJ3KvyPXdSmZt6cU6%2Flanguage-model.png?generation=1586576265109629&alt=media)


In [3]:
#"hello"를 one-hot encoding을 사용하여 벡터로 치환해봅시다
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [4]:
input_data_np = np.array([[h, e, l, l, o]], dtype=np.float32)
input_data = torch.Tensor(input_data_np)
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]])

- 앞에서 선언한 벡터를 이용하여 추가로, "eolll", "lleel"를 벡터로 치환해봅시다

In [5]:
# transform as torch tensor
input_data_np = np.array([[h, e, l, l, o],
                          [e, o, l, l, l],
                          [l, l, e, e, l]], dtype=np.float32)
input_data = torch.Tensor(input_data_np)
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])

In [6]:
# (batch size, seq length(time step), input_size)
input_data.size()

torch.Size([3, 5, 4])

## 1. RNN 계열 모델 정의
1. Vanila RNN
2. LSTM




### 2.1 Vanila RNN

#### `torch.nn.RNN(input_size, hidden_size, batch_first)`
- [pytorch document](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
- `input_size`: 입력해 주는 특성 값의 개수
- `hidden_size`: hidden state의 사이즈 지정. 보통 arbitrary 함.
- `batch_first`: 입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정.
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)

In [7]:
# declare dimension
input_size = 4
hidden_size = 3

# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)  # 처음에 batch_size or seq_length

In [8]:
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])

In [9]:
# check output
outputs, _status = rnn(input_data)

print("--------------모든 time steps의 hidden states-----------------")
print("outputs:\n", outputs)
print("outputs size:",outputs.size()) # (5번의 time step동안 3차원의 은닉 상태가 출력됨)*단어 3개
# shape : (3, 5, 3)
# (batch_size, seq_length, hidden_dim)
print("--------------마지막 시점의 hidden state-----------------")
print("hidden state:\n", _status) # 마지막 시점의 hidden state
print("hidden state size:",_status.size())
# shape: (1, 3, 3)
# (num_layers, batch_size, hidden_dim)

--------------모든 time steps의 hidden states-----------------
outputs:
 tensor([[[-0.3347, -0.2007, -0.5535],
         [-0.0397,  0.3203,  0.0904],
         [ 0.3053, -0.7721, -0.2728],
         [ 0.4136, -0.7323, -0.1871],
         [ 0.3318, -0.3053,  0.3274]],

        [[-0.2411, -0.1549,  0.2371],
         [ 0.0951, -0.2110,  0.2179],
         [ 0.2171, -0.8062, -0.1927],
         [ 0.3664, -0.7264, -0.1921],
         [ 0.3913, -0.7672, -0.1641]],

        [[ 0.3230, -0.7496, -0.2588],
         [ 0.4118, -0.7411, -0.1825],
         [-0.1657, -0.2229,  0.3397],
         [-0.4297, -0.2101,  0.2656],
         [ 0.1204, -0.6876, -0.2978]]], grad_fn=<TransposeBackward1>)
outputs size: torch.Size([3, 5, 3])
--------------마지막 시점의 hidden state-----------------
hidden state:
 tensor([[[ 0.3318, -0.3053,  0.3274],
         [ 0.3913, -0.7672, -0.1641],
         [ 0.1204, -0.6876, -0.2978]]], grad_fn=<StackBackward0>)
hidden state size: torch.Size([1, 3, 3])


## 2.2 LSTM


#### `torch.nn.LSTM(input_size, hidden_size, batch_first)`
- [pytorch document](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
- `input_size`: 입력해 주는 특성 값의 개수
- `hidden_size`: hidden state의 사이즈 지정. 보통 arbitrary 함.
- `batch_first`: 입력으로 받는 데이터의 shape중 첫 번째 차원을 batch로 간주할 것인지를 설정.
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)

In [10]:
lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
outputs, _status = lstm(input_data)
print(outputs)
print(outputs.size())
print()

print(_status[0])
print(_status[0].shape)

tensor([[[-0.0907, -0.1976, -0.1218],
         [-0.1799, -0.3090,  0.0233],
         [-0.1810, -0.1710, -0.1549],
         [-0.1883, -0.1357, -0.2348],
         [-0.1956, -0.1958, -0.2009]],

        [[-0.1528, -0.2698,  0.1212],
         [-0.1743, -0.2623, -0.1078],
         [-0.1776, -0.1333, -0.2285],
         [-0.1831, -0.1068, -0.2705],
         [-0.1816, -0.0881, -0.2921]],

        [[-0.1260, -0.0651, -0.1560],
         [-0.1645, -0.0830, -0.2329],
         [-0.2112, -0.2959, -0.0419],
         [-0.2231, -0.3255,  0.0465],
         [-0.1980, -0.1818, -0.1475]]], grad_fn=<TransposeBackward0>)
torch.Size([3, 5, 3])

tensor([[[-0.1956, -0.1958, -0.2009],
         [-0.1816, -0.0881, -0.2921],
         [-0.1980, -0.1818, -0.1475]]], grad_fn=<StackBackward0>)
torch.Size([1, 3, 3])
